In [1]:

###
import timm
import copy
from tqdm import tqdm
from PIL import Image
from fastprogress import master_bar, progress_bar
from torchtoolbox.tools import mixup_data, mixup_criterion
from torch.autograd import Variable
from torchvision import models
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import time
import sys
from pathlib import Path
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
import warnings
warnings.filterwarnings("ignore")
####
####
##

/home/csl/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class ChestDataset(Dataset):
    def __init__(self, csv_file, dir, transform=None):
        df = pd.read_csv(csv_file)
        df.fillna(0, inplace=True)  # Nan 0으로 간주
        for j in range(len(df)):
            for i in range(5, 19):
                if df.iloc[j, i] < 0:
                    df.iloc[j, i] = 0    # -1 - 0으로 간주

        self.data = df
        self.dir = dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.dir+self.data.iloc[idx, 0]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        y = self.data.iloc[idx, 5:19]
        y = torch.tensor(y, dtype=torch.float32)
        return image, y

class ChestTest(Dataset):
    def __init__(self, csv_file, dir, transform=None):
        df = pd.read_csv(csv_file)

        self.data = df
        self.dir = dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.dir+self.data.iloc[idx, 0]
        image = Image.open(image_path)
        image = self.transform(image)
        y = self.data.iloc[idx, 1:]
        y = torch.tensor(y, dtype=torch.float32)
        return image, y

class ResNet50(nn.Module):
    def __init__(self, num_classes=14, is_trained=False):

        super().__init__()
        self.net = timm.create_model('resnet50', pretrained=True)

        # Replace last layer with new layer that have num_classes nodes, after that apply Sigmoid to the output
        self.net.fc = nn.Linear(2048, 14)

    def forward(self, inputs):
        return self.net(inputs)

In [11]:
cols = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
            'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
            'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
            'Fracture', 'Support Devices']
test_scores_list = []

@torch.no_grad()
def test_model(model, test_loader, criterion, device):
    model.eval()
    all_preds = []
    all_labels = []

    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        preds = model(data)
        all_preds.append(preds)
        all_labels.append(labels)

    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    test_loss = criterion(all_preds, all_labels).item()

    all_preds = all_preds.cpu().numpy()
    all_labels = all_labels.cpu().numpy()

    # calculate accuracy per label
    # calculate auc_roc per label
    # calculate precision, recall, f1 score per label
    scores = []
    metrics=[]
    for i in range(14):
        # count true positive, true negative, false positive, false negative
        metrics.append([np.sum((np.array(all_labels[:, i])==1) & (np.array(all_preds[:, i])>0.7)),
                        np.sum((np.array(all_labels[:, i])==0) & (np.array(all_preds[:, i])<0.7)),
                        np.sum((np.array(all_labels[:, i])==0) & (np.array(all_preds[:, i])>0.7)),
                        np.sum((np.array(all_labels[:, i])==1) & (np.array(all_preds[:, i])<0.7))])
        
        accuracy = accuracy_score(all_labels[:, i], all_preds[:, i] > 0.7)
        auc_roc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        precision = precision_score(
            all_labels[:, i], all_preds[:, i] > 0.7)
        recall = recall_score(all_labels[:, i], all_preds[:, i] > 0.7)
        f1 = f1_score(all_labels[:, i], all_preds[:, i] > 0.7)
        scores.append([accuracy, auc_roc, precision, recall, f1])

    scores = pd.DataFrame(
        scores, columns=['Accuracy', 'AUC-ROC', 'Precision', 'Recall', 'F1 Score'])
    test_scores_list.append(scores)
    print(scores)

    # save the print to a file

    # with open('ResNet50_Test_Log.txt', 'a') as f:
    #     print(scores, file=f)

    print('Test Loss: {:.4f}'.format(test_loss))
    
    metrics=pd.DataFrame(metrics, columns=['TP', 'TN', 'FP', 'FN'])
    print(metrics)

    # save the print to a file

    # with open('ResNet50_Test_Log.txt', 'a') as f:
    #     print('Test Loss: {:.4f}'.format(test_loss), file=f)

    # plot auc_roc curve for each label
    # for i in range(14):
    #     fpr, tpr, _ = roc_curve(all_labels[:, i], all_preds[:, i])
    #     plt.figure()
    #     plt.plot(fpr, tpr)
    #     plt.xlabel('False Positive Rate')
    #     plt.ylabel('True Positive Rate')
    #     plt.title('ROC Curve for Label {}'.format(cols[i]))
    #     plt.savefig('ResNet50_ROC_Curve_Label_{}.png'.format(cols[i]))

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.cuda.empty_cache()

print(device)

base_path = os.getcwd()+'/'
print(base_path)

BATCH_SIZE = 96
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.472, 0.472, 0.472], [0.320, 0.320, 0.320]),])

test_data = ChestTest(base_path+'test_labels.csv',
                      base_path, transform=valid_transform)
print("data")

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)
print("load")

cuda
/home/csl/jhb/
data
load


# ResNet Result

In [13]:
model = ResNet50()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    # model=nn.DataParallel(model)
model.to(device)

ResNet50(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentu

In [6]:
criterion = nn.MultiLabelSoftMarginLoss()

In [14]:
model.load_state_dict(torch.load(base_path+'ResNet50_best.pth'))

<All keys matched successfully>

In [17]:
test_model(model, test_loader, criterion, device)

    Accuracy   AUC-ROC  Precision    Recall  F1 Score
0   0.844311  0.886380   0.608696  0.128440  0.212121
1   0.553892  0.586133   0.000000  0.000000  0.000000
2   0.751497  0.869812   1.000000  0.051429  0.097826
3   0.643713  0.904983   0.961538  0.241935  0.386598
4   0.979042  0.910004   0.000000  0.000000  0.000000
5   0.881737  0.899385   0.615385  0.188235  0.288288
6   0.949102  0.891266   1.000000  0.028571  0.055556
7   0.979042  0.677698   0.000000  0.000000  0.000000
8   0.735030  0.826531   1.000000  0.005618  0.011173
9   0.985030  0.935410   0.500000  0.300000  0.375000
10  0.914671  0.954075   0.805825  0.691667  0.744395
11  0.988024  0.967045   0.000000  0.000000  0.000000
12  0.992515  0.718530   1.000000  0.166667  0.285714
13  0.820359  0.939584   0.966507  0.641270  0.770992
Test Loss: 0.3290
     TP   TN  FP   FN
0    14  550   9   95
1     0  370   0  298
2     9  493   0  166
3    75  355   3  235
4     0  654   0   14
5    16  573  10   69
6     1  633   0  

# EfficientNet

In [52]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes=14, is_trained=False):

        super().__init__()
        # efficientnet_b0
        self.net = timm.create_model(
            'efficientnet_b0', pretrained=False, num_classes=14)

    def forward(self, inputs):
        return self.net(inputs)

In [55]:
model = EfficientNet()
model.to(device)
model.load_state_dict(torch.load(base_path+'EfficientNet_best.pth'))

<All keys matched successfully>

In [56]:
test_model(model, test_loader, criterion, device)

    Accuracy   AUC-ROC  Precision    Recall  F1 Score
0   0.833832  0.885428   0.416667  0.045872  0.082645
1   0.553892  0.495474   0.000000  0.000000  0.000000
2   0.750000  0.832141   1.000000  0.045714  0.087432
3   0.684132  0.897432   0.938053  0.341935  0.501182
4   0.980539  0.897444   1.000000  0.071429  0.133333
5   0.869760  0.882716   0.444444  0.094118  0.155340
6   0.947605  0.880975   0.000000  0.000000  0.000000
7   0.979042  0.664701   0.000000  0.000000  0.000000
8   0.733533  0.726886   0.000000  0.000000  0.000000
9   0.983533  0.933131   0.000000  0.000000  0.000000
10  0.899701  0.945742   0.734513  0.691667  0.712446
11  0.988024  0.960985   0.000000  0.000000  0.000000
12  0.991018  0.657351   0.000000  0.000000  0.000000
13  0.821856  0.917640   0.949541  0.657143  0.776735
Test Loss: 0.3417
     TP   TN  FP   FN
0     5  552   7  104
1     0  370   0  298
2     8  493   0  167
3   106  351   7  204
4     1  654   0   13
5     8  573  10   77
6     0  633   0  

# SEResNet50

In [18]:
class SEResNet50(nn.Module):
    def __init__(self, num_classes=14, is_trained=False):

        super().__init__()
        self.net = timm.create_model(
            'seresnet50', pretrained=True, num_classes=14)


    def forward(self, inputs):
        return self.net(inputs)

In [19]:
model=SEResNet50()
model.to(device)
model.load_state_dict(torch.load(base_path+'SEResNet50_best.pth'))

<All keys matched successfully>

In [20]:
test_model(model, test_loader, criterion, device)

    Accuracy   AUC-ROC  Precision    Recall  F1 Score
0   0.841317  0.891254   0.636364  0.064220  0.116667
1   0.553892  0.641067   0.000000  0.000000  0.000000
2   0.747006  0.862834   1.000000  0.034286  0.066298
3   0.666168  0.888547   0.957895  0.293548  0.449383
4   0.979042  0.888707   0.000000  0.000000  0.000000
5   0.874251  0.881566   0.520000  0.152941  0.236364
6   0.947605  0.871316   0.000000  0.000000  0.000000
7   0.979042  0.669179   0.000000  0.000000  0.000000
8   0.733533  0.778021   0.000000  0.000000  0.000000
9   0.986527  0.872188   0.666667  0.200000  0.307692
10  0.907186  0.952676   0.719697  0.791667  0.753968
11  0.988024  0.970455   0.000000  0.000000  0.000000
12  0.991018  0.711480   0.000000  0.000000  0.000000
13  0.842814  0.937101   0.972973  0.685714  0.804469
Test Loss: 0.3369
     TP   TN  FP   FN
0     7  555   4  102
1     0  370   0  298
2     6  493   0  169
3    91  354   4  219
4     0  654   0   14
5    13  571  12   72
6     0  633   0  